<a href="https://colab.research.google.com/github/freedomtan/clip_score_on_android/blob/main/verify_decoder_fp16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q diffusers
!pip install -q accelerate
!pip install -q keras_cv

In [2]:
from diffusers import StableDiffusionPipeline
import torch

import keras_cv
import keras

import tensorflow as tf
import math
import numpy as np

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
# pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to("cuda")

torch_text_encoder = pipe.text_encoder

keras.mixed_precision.set_global_policy("mixed_float16")
pipeline = keras_cv.models.StableDiffusion(
    img_width=512, img_height=512, jit_compile=True
)

Using TensorFlow backend


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [3]:
for i in range(10):
  # r_1 is for the keras vae decoder
  # r_2 is NCHW of r_1 for PyTorch vae decoder
  r_1 = tf.random.stateless_normal((1, 64, 64, 4), seed=[i, i])
  r_2 = tf.transpose(r_1, perm=[0, 3, 1, 2])
  r_2_fp16 = torch.from_numpy(r_2.numpy()).to("cuda").type(torch.float16)

  tf_decoded = pipeline.decoder(r_1)
  tf_decoded_uint8 = tf.cast(((tf_decoded + 1) / 2) * 255, tf.uint8)

  # the input tensor is divided pipe.vae.config.scaling_factor before sending to the vae decoder
  decoded = pipe.vae.decode(r_2_fp16 * 1 / pipe.vae.config.scaling_factor, return_dict=False)
  torch_decoded_uint8 =  np.uint8(np.transpose((decoded[0].cpu().detach().numpy() / 2 + 0.5), (0, 2, 3, 1)) * 255)

  print(tf_decoded_uint8 - torch_decoded_uint8)

tf.Tensor(
[[[[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0 255   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  1   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   4   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  ...

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [  0   0   0]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]

  [[  0   0   0]
   [255   0   1]
   [  0   0   0]
   ...
   [  0   0   0]
   [  0   0   0]
   [  0   0   0]]]], shape=(1, 512, 512, 3), dtype=uint8)
tf.Tensor(
[[[[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 0]
   [0 0 0]
   [0 0 0]]

  [[0 0 0]
   [0 0 0]
   [0 0 0]
   ...
   [0 0 1]
   [0 0 0]
   [0 0 0]]

  ...

  [[0 0 0]
   [0 0 0]
   [1 0 0]
 